In [1]:
import os, sys, email
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *
import pprint, re, time, pickle

pd.options.display.max_colwidth = 1000

In [2]:
### 自然言語処理
import spacy 
nlp = spacy.load("en_core_web_sm")

from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()
nlp.add_pipe(sentencizer)

from spacy.lemmatizer import Lemmatizer
from spacy.lookups import Lookups
lookups = Lookups()
lemmatizer = Lemmatizer(lookups)

In [3]:
# loadする

with open('/Users/taroaso/myprojects/OpenIE/trec/output/entity_dict.pickle', mode='rb') as f:
    entity_dict = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/entity_rows.pickle', mode='rb') as f:
    entity_rows = pickle.load(f)

In [4]:
from db import connect
engine = connect()
mail_df = pd.read_sql(sql='SELECT * FROM mail_3 ORDER BY docno ASC',con=engine, index_col=None)

In [5]:
# Bodyに含まれる文をOpen IEにかけてTripleを抽出する．
count = 0
sentence_list = []
pattern = re.compile('\W+')

for mail in mail_df.values:
    count = count + 1
    body = mail[-1]
    body = ' '.join(body.split())
    doc = nlp(body)

    # 記号や数字だけのsentenceや閾値以下／以上のトークン数のsentenceをフィルタリングする
    for sentence in doc.sents:
        tokens = sentence.text.split()
        sentence = ' '.join(tokens)
        sentence = re.sub(r'[>|<]{1,}','',sentence)
        sentence = re.sub(r' {2,}',' ',sentence)
        if len(sentence) == 0 or sentence.isnumeric() or pattern.fullmatch(sentence) or len(sentence.split()) <= 5:
            pass
        else:
            sentence_list.append((mail[0],sentence))

In [19]:
total_start = time.time()
# MinIEにかける
import requests
import json

count = 0
extractions_list = []

for tpl in sentence_list:
    count = count + 1
    message_id = tpl[0]
    sentence = tpl[1].encode('utf-8')
    try:
        response = requests.post('http://localhost:8080/minie/query', data=sentence)
        result = response.json()
        if result['facts'] == []:
            pass
        else:
            for triple in result['facts']:
                sbj = triple['subject']
                obj = triple['object']
                predicate = triple['predicate']
                extractions_list.append([message_id, sentence.decode('utf-8'), sbj, predicate, obj])
            pprint.pprint('c{}/{}'.format(count,len(sentence_list)))
    except json.JSONDecodeError:
        pass

total_time = time.time() - total_start
print("Total time:{} minutes".format(total_time/60))

&#39;c3/10725&#39;
&#39;c7/10725&#39;
&#39;c8/10725&#39;
&#39;c11/10725&#39;
&#39;c12/10725&#39;
&#39;c16/10725&#39;
&#39;c19/10725&#39;
&#39;c22/10725&#39;
&#39;c23/10725&#39;
&#39;c24/10725&#39;
&#39;c25/10725&#39;
&#39;c26/10725&#39;
&#39;c27/10725&#39;
&#39;c28/10725&#39;
&#39;c30/10725&#39;
&#39;c31/10725&#39;
&#39;c32/10725&#39;
&#39;c33/10725&#39;
&#39;c34/10725&#39;
&#39;c35/10725&#39;
&#39;c36/10725&#39;
&#39;c37/10725&#39;
&#39;c38/10725&#39;
&#39;c39/10725&#39;
&#39;c43/10725&#39;
&#39;c45/10725&#39;
&#39;c46/10725&#39;
&#39;c47/10725&#39;
&#39;c50/10725&#39;
&#39;c52/10725&#39;
&#39;c54/10725&#39;
&#39;c55/10725&#39;
&#39;c66/10725&#39;
&#39;c67/10725&#39;
&#39;c68/10725&#39;
&#39;c69/10725&#39;
&#39;c70/10725&#39;
&#39;c71/10725&#39;
&#39;c75/10725&#39;
&#39;c78/10725&#39;
&#39;c79/10725&#39;
&#39;c82/10725&#39;
&#39;c84/10725&#39;
&#39;c89/10725&#39;
&#39;c93/10725&#39;
&#39;c96/10725&#39;
&#39;c97/10725&#39;
&#39;c101/10725&#39;
&#39;c102/10725&#39;
&#39;c125/10725&#39;


In [23]:
# dataframeにする
triple_df = pd.DataFrame(extractions_list, columns = ['docno','sentence', 'subject', 'predicate', 'object'])

In [26]:
translate_dict = {}
for value in mail_df.values:
    translate_dict[value[0]]=value[5]

In [44]:
triples = []
for value in triple_df.values:
    docno = value[0]
    sentence = value[1]
    subj = value[2]
    pred = value[3]
    obje = value[4]
    if subj in entity_dict:
        new_subj = entity_dict[subj].keys()
        new_subj = list(new_subj)[0]
        triples.append([docno,sentence,new_subj,pred,obje])

In [52]:
triples2 = []
for value in triples:
    docno = value[0]
    sentence = value[1]
    subj = value[2]
    pred = value[3]
    obje = value[4]
    if obje in entity_dict:
        new_obje = entity_dict[obje].keys()
        new_obje = list(new_obje)[0]
        triples2.append([docno,sentence,subj,pred,new_obje])

In [57]:
subject_entity = []
for triple in triples:
    subject_entity.append(triple[2])
len(set(subject_entity))

100

In [59]:
pprint.pprint(set(subject_entity))

{&#39;12 Sep 2002&#39;,
 &#39;15 Feb 2000&#39;,
 &#39;19 Jun 2002&#39;,
 &#39;30 Nov 2000&#39;,
 &#39;Access key&#39;,
 &#39;AlltheWeb&#39;,
 &#39;Alt attribute&#39;,
 &#39;AltaVista&#39;,
 &#39;Amaya (web editor)&#39;,
 &#39;Apache HTTP Server&#39;,
 &#39;Batik (software)&#39;,
 &#39;Benjamin Franklin&#39;,
 &#39;Berkeley Square, Bristol&#39;,
 &#39;Bernd Marscheider&#39;,
 &#39;Carol Foster&#39;,
 &#39;Cascading Style Sheets&#39;,
 &#39;Charles McCathieNevile&#39;,
 &#39;Colin Newman&#39;,
 &#39;Content negotiation&#39;,
 &#39;Dan Brickley&#39;,
 &#39;Dan Connolly&#39;,
 &#39;Dave Beckett&#39;,
 &#39;David Poehlman&#39;,
 &#39;Didcot&#39;,
 &#39;Domain Name System&#39;,
 &#39;E-commerce&#39;,
 &#39;Email&#39;,
 &#39;Eric Miller (rugby union)&#39;,
 &#39;Gmail&#39;,
 &#39;Google&#39;,
 &#39;Google Images&#39;,
 &#39;Google Search&#39;,
 &#39;Google Toolbar&#39;,
 &#39;Gregory J. Rosmaita&#39;,
 &#39;Guide dog&#39;,
 &#39;HTML&#39;,
 &#39;HTML element&#39;,
 &#39;Hyperlink&#39;,
 &#39;

In [46]:
entity_list = []
for entity in entity_dict.values():
    entity = entity.keys()
    entity = list(entity)[0]
    entity_list.append(entity)
entity_set = set(entity_list)
len(entity_set)

1135

In [58]:
mail_df.head()

,docno,received,isoreceived,sent,isosent,name,email,subject,id,charset,inreplyto,expires,to,cc,body
0,lists-000-13503992,Fri Jul 14 12:48:10 2000,20000714164810,"Fri, 14 Jul 2000 08:19:41 -0400 (EDT)",20000714121941,Georgopoulos N,ngeorgopoulos@city.ac.uk,Bug report,E13D4Rs-0004TB-00@mailswitch,US-ASCII,None,-1,html-tidy@w3.org,None,\n\n\n\n\n\n\n\n\n\ntext/enriched attachment: stored\n\n\n\n\n\n\n\n\n
1,lists-001-9920622,Sat Jul 14 07:24:02 2001,20010714112402,"Sat, 14 Jul 2001 13:12:58 +0200",20010714111258,Karl Ove Hufthammer,huftis@bigfoot.com,Re: entities in title tags,003b01c10c55$f0f62060$f3468ed5@huftis,iso-8859-1,000201c10c12$b894f900$4222e540&#64;IanEvans,-1,"""Ian M. Evans""<ianevans@digitalhit.com>,<html-tidy@w3.org>",None,"\n\n\n\n\n\n----- Original Message ----- \n\nFrom: ""Ian M. Evans"" <ianevans@digitalhit.com>\n\nSent: Saturday, July 14, 2001 5:11 AM\n\nSubject: entities in title tags\n\n\n\n\n\n> Just wondering if there's anything wrong with entities in title tags.\n\n\n\nNo.\n\n\n\n> I've noticed that the numeric entity for the apostrophe ends up showing up\n\n> in Google search results as:\n\n\n\nThis is a bug in Google.\n\n\n\n-- \n\nKarl Ove Hufthammer\n\n\n\n\n\n\n"
2,lists-002-10134409,Tue Apr 29 10:23:48 2003,20030429142348,"Tue, 29 Apr 2003 22:23:41 +0800",20030429142341,lock mode,klcc70@hotmail.com,"Stock markets rise on positive earnings, consumer spending ConsumerReportsorg",Law10-F458juXym3SjK00016b82@hotmail.com,None,None,-1,html-tidy@w3.org,None,"\n\n\n\n\n\n\n\nUnbiased product Ratings from the experts at Product Ratings, product \n\nreviews, buying guides, product safety recalls and consumer information from \n\nthe experts at Consumer Union/Consumer Reports Magazine. Information, \n\nratings, and advice on products, services, and decisions. new car reviews, \n\nused car Consumer Reports Cars and Trucks: Stay informed with our latest new \n\ncar reviews, used car reviews, auto ratings, car safety reports and auto \n\nprice service. Provides car reviews, automobile safety information, car \n\nbuying guidance.\n\n\n\n\n\n\n\nhttp://www.geocities.com/webrobotic2/ConsumerReports.htm\n\n\n\n\n\nconsumer report\n\n12965 consumer report magazine\n\n11859 consumer report car\n\n9172 consumer report online\n\n7784 consumer report auto\n\n4280 consumer report vacuum cleaners\n\n3970 consumer report digital camera\n\n3764 consumer credit report\n\n3633 consumer report automobile\n\n2817 free consumer report\n\n2477 consume..."
3,lists-002-10233238,Sat May 17 03:24:47 2003,20030517072447,"Wed, 14 May 2003 09:32:46 -0400 (EDT)",20030514133246,David Van Cott,dtvc17@msn.com,Tag problem,BAY4-F144h4Jzgi1Z8200004355@hotmail.com,None,None,-1,html-tidy@w3.org,dsr@w3.org,\n\n\n\n\n\n\n\n\n\n\n\n\n\nI'm using html tidy at work to make the website more accessible and I'm \n\nhaving a little problem. Everything seems to work great execept for one \n\nthing. Tidy seems to be removing the <dl> tags. This is causing the \n\nalignment of the webpages to be off. I searched through google newsgroups \n\nand the www.w3.org site and I couldnt find any questions or solutions \n\nregarding this. There is my configs...\n\n--clean yes\n\n--output-xml no\n\n--enclose-text yes\n\n--enclose-block-text yes\n\n--indent auto\n\n--alt-text GraphicImage\n\n\n\nOther then that problem I think this program is great. I have to make sure \n\nover 1000 html files have alt attributes inside the img tag. This make file \n\nmuch similar. Thanks\n\nDavid\n\n\n\n_________________________________________________________________\n\nHelp STOP SPAM with the new MSN 8 and get 2 months FREE* \n\nhttp://join.msn.com/?page=features/junkmail\n\n\n\n\n\n\n
4,lists-002-15997816,Sat Nov 22 17:30:35 2003,20031122223035,"Wed, 19 Nov 2003 18:42:25 -0800",20031120024225,Azigan,azigan@fix.net,www.worldwidewoodworking would like to exchange links,0HOM00LVHQ68TB@a34-mta02.direcway.com,None,None,-1,Html Tidy<html-tidy@w3.org>,None,"\n\n\n\n\n\n\n\nWe both know the 